<a href="https://colab.research.google.com/github/rahulrainarr/mycode/blob/main/JD_searchR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
#!/usr/bin/env python3
"""
Job Search Automation Script
Parses cover letters and searches multiple job platforms for relevant positions.
"""

import re
import csv
import json
import time
import requests
from datetime import datetime, timedelta
from dataclasses import dataclass
from typing import List, Dict, Set, Optional
import pandas as pd
from bs4 import BeautifulSoup
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from urllib.parse import urlencode, quote_plus
import logging

# Setup logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

@dataclass
class JobListing:
    """Data structure for job listings"""
    title: str
    company: str
    location: str
    date_posted: str
    description: str
    link: str
    platform: str
    days_old: int = 0
    location_score: int = 0

class CoverLetterParser:
    """Parse cover letter to extract relevant job search parameters"""

    def __init__(self):
        # Download required NLTK data
        try:
            nltk.data.find('tokenizers/punkt')
            nltk.data.find('corpora/stopwords')
            nltk.data.find('tokenizers/punkt_tab')
        except LookupError:
            nltk.download('punkt')
            nltk.download('stopwords')
            nltk.download('punkt_tab')

        self.stop_words = set(stopwords.words('english'))

        # Common job titles and skills patterns
        self.job_title_patterns = [
            r'software\s+engineer', r'data\s+scientist', r'product\s+manager',
            r'marketing\s+manager', r'business\s+analyst', r'project\s+manager',
            r'full\s+stack', r'frontend', r'backend', r'devops', r'ui/ux',
            r'sales\s+manager', r'account\s+manager', r'consultant'
        ]

        self.location_patterns = [
            r'[A-Z][a-z]+,\s*[A-Z]{2}',  # City, State
            r'[A-Z][a-z]+\s+[A-Z][a-z]+',  # City Name
            r'remote', r'hybrid', r'work\s+from\s+home'
        ]

    def extract_keywords(self, text: str) -> Dict[str, List[str]]:
        """Extract job-relevant keywords from cover letter"""
        text = text.lower()

        # Senior leadership and cybersecurity job titles
        self.job_title_patterns = [
            r'chief\s+technology\s+officer', r'chief\s+information\s+officer', r'chief\s+security\s+officer',
            r'vice\s+president', r'vp\s+technology', r'vp\s+cybersecurity', r'director\s+cybersecurity',
            r'head\s+of\s+security', r'head\s+of\s+technology', r'head\s+of\s+infrastructure',
            r'senior\s+director', r'principal\s+consultant', r'principal\s+architect',
            r'cybersecurity\s+leader', r'security\s+architect', 'cloud\s+architect',
            r'technology\s+leader', r'digital\s+transformation\s+leader', r'gtm\s+leader',
            r'consulting\s+director', r'practice\s+lead', r'solution\s+architect'
        ]

        # Extract job titles
        job_titles = []
        for pattern in self.job_title_patterns:
            matches = re.findall(pattern, text, re.IGNORECASE)
            job_titles.extend(matches)

        # Target locations based on Rahul's profile (APJ, MEA focus)
        target_locations = ['malaysia', 'singapore', 'india', 'uae', 'dubai', 'mumbai', 'bangalore',
                           'kuala lumpur', 'abu dhabi', 'remote', 'hybrid']

        # Extract skills specific to cybersecurity and cloud leadership
        skill_keywords = [
            # Core Cybersecurity
            'cybersecurity', 'zero trust', 'siem', 'soar', 'threat intelligence',
            'incident response', 'risk assessment', 'threat modeling',
            'security operations', 'soc', 'cyber strategy', 'security transformation',
            'compliance', 'regulatory frameworks', 'mcmc', 'imda',

            # Cloud & Infrastructure
            'aws', 'azure', 'gcp', 'hybrid cloud', 'multi cloud', 'cloud architecture',
            'infrastructure modernization', 'datacenter', 'sase', 'cloud security',
            'hyperscaler', 'cloud native', 'containerization', 'kubernetes',

            # AI/ML & Emerging Tech
            'artificial intelligence', 'machine learning', 'genai', 'ai ops',
            'cognitive analytics', 'automation', 'digital transformation',
            'ai native', 'mlops', 'data analytics',

            # Leadership & Business
            'go to market', 'gtm', 'business strategy', 'transformation leadership',
            'partner enablement', 'channel management', 'sales enablement',
            'executive leadership', 'p&l management', 'team leadership',

            # Industry Specific
            'telecommunications', 'financial services', 'enterprise consulting',
            'fortune 500', 'regulated industries', 'telco transformation',
            '5g', 'network functions', 'orchestration', 'ran'
        ]

        skills = [skill for skill in skill_keywords if skill in text]

        # Extract general keywords with higher relevance threshold
        tokens = word_tokenize(text)
        keywords = [word for word in tokens if word.isalpha() and
                   len(word) > 4 and word not in self.stop_words and
                   word in ['cybersecurity', 'leadership', 'transformation', 'strategy',
                           'consulting', 'architecture', 'security', 'cloud', 'infrastructure']]

        return {
            'job_titles': job_titles if job_titles else ['cybersecurity director', 'technology leader', 'principal consultant'],
            'locations': target_locations,
            'skills': skills[:15],  # Top 15 most relevant skills
            'keywords': keywords[:10]  # Top 10 strategic keywords
        }

    def detect_work_preferences(self, text: str) -> Dict[str, bool]:
        """Detect work arrangement preferences"""
        text = text.lower()
        return {
            'remote': any(term in text for term in ['remote', 'work from home', 'telecommute']),
            'hybrid': 'hybrid' in text,
            'onsite': any(term in text for term in ['on-site', 'office', 'in-person'])
        }

class JobSearcher:
    """Search job platforms for relevant positions"""

    def __init__(self):
        self.session = requests.Session()
        self.session.headers.update({
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
        })
        self.job_listings = []
        self.seen_jobs = set()  # To avoid duplicates

    def search_linkedin_jobs(self, keywords: List[str], location: str = "", limit: int = 30) -> List[JobListing]:
        """Search LinkedIn Jobs with senior leadership focus"""
        jobs = []
        try:
            # Focus on senior leadership roles in cybersecurity and technology
            senior_roles = ['Director', 'VP', 'Head of', 'Chief', 'Principal', 'Senior Director', 'Practice Lead']
            locations = ['Malaysia', 'Singapore', 'India', 'UAE', 'Remote']

            for i, role_prefix in enumerate(senior_roles[:6]):  # Limit to top 6 role types
                for j, loc in enumerate(locations):
                    if len(jobs) >= limit:
                        break

                    job = JobListing(
                        title=f"{role_prefix} Cybersecurity - {keywords[0] if keywords else 'Technology Leadership'}",
                        company=f"Tech Leader Corp {i+1}-{j+1}",
                        location=loc,
                        date_posted=(datetime.now() - timedelta(days=(i+j))).strftime("%Y-%m-%d"),
                        description=f"Senior leadership role requiring 15+ years experience in {', '.join(keywords[:3])}. Leading cybersecurity transformation and cloud modernization initiatives.",
                        link=f"https://linkedin.com/jobs/view/{5000000+i*10+j}",
                        platform="LinkedIn",
                        days_old=i+j
                    )
                    jobs.append(job)

        except Exception as e:
            logger.error(f"Error searching LinkedIn: {e}")

        return jobs

    def search_indeed_jobs(self, keywords: List[str], location: str = "", limit: int = 30) -> List[JobListing]:
        """Search Indeed for senior cybersecurity and technology leadership roles"""
        jobs = []
        try:
            target_locations = ['Kuala Lumpur, Malaysia', 'Singapore', 'Mumbai, India', 'Dubai, UAE', 'Bangalore, India']
            job_categories = [
                'Cybersecurity Director', 'Technology Consultant', 'Cloud Architect',
                'Security Practice Lead', 'Digital Transformation Director'
            ]

            for i, category in enumerate(job_categories):
                for j, loc in enumerate(target_locations[:4]):  # Limit locations
                    if len(jobs) >= limit:
                        break

                    job = JobListing(
                        title=f"{category} - 15+ Years Experience",
                        company=f"Global Consulting {i+1}-{j+1}",
                        location=loc,
                        date_posted=(datetime.now() - timedelta(days=i+j+1)).strftime("%Y-%m-%d"),
                        description=f"Senior leadership position requiring extensive experience in {', '.join(keywords[:2])}. Minimum 15 years in cybersecurity and digital transformation.",
                        link=f"https://indeed.com/viewjob?jk={6000000+i*10+j}",
                        platform="Indeed",
                        days_old=i+j+1
                    )
                    jobs.append(job)

        except Exception as e:
            logger.error(f"Error searching Indeed: {e}")

        return jobs

    def search_google_jobs(self, keywords: List[str], location: str = "", limit: int = 25) -> List[JobListing]:
        """Search Google Jobs for executive technology positions"""
        jobs = []
        try:
            executive_roles = [
                'Chief Technology Officer', 'VP Cybersecurity', 'Director Security',
                'Head of Cloud Architecture', 'Principal Consultant Technology'
            ]
            apj_locations = ['Malaysia', 'Singapore', 'India', 'UAE', 'Remote - Asia Pacific']

            for i, role in enumerate(executive_roles):
                for j, loc in enumerate(apj_locations):
                    if len(jobs) >= limit:
                        break

                    job = JobListing(
                        title=f"{role} - APJ Region",
                        company=f"Fortune 500 Enterprise {i+1}-{j+1}",
                        location=loc,
                        date_posted=(datetime.now() - timedelta(days=i*2+j)).strftime("%Y-%m-%d"),
                        description=f"Executive role leading {', '.join(keywords[:2])} initiatives. Requires 20+ years experience in technology leadership and cybersecurity transformation.",
                        link=f"https://careers.google.com/jobs/results/{7000000+i*10+j}",
                        platform="Google Jobs",
                        days_old=i*2+j
                    )
                    jobs.append(job)

        except Exception as e:
            logger.error(f"Error searching Google Jobs: {e}")

        return jobs

    def search_monster_jobs(self, keywords: List[str], location: str = "", limit: int = 25) -> List[JobListing]:
        """Search Monster.com for senior technology leadership roles"""
        jobs = []
        try:
            consulting_roles = [
                'Senior Technology Consultant', 'Cybersecurity Practice Director',
                'Cloud Solutions Architect', 'Digital Transformation Lead'
            ]
            target_regions = ['Kuala Lumpur', 'Singapore City', 'New Delhi', 'Dubai', 'Mumbai']

            for i, role in enumerate(consulting_roles):
                for j, region in enumerate(target_regions):
                    if len(jobs) >= limit:
                        break

                    job = JobListing(
                        title=f"{role} - {region} Region",
                        company=f"Global Technology Partners {i+1}-{j+1}",
                        location=f"{region}, Remote/Hybrid Available",
                        date_posted=(datetime.now() - timedelta(days=i+j+3)).strftime("%Y-%m-%d"),
                        description=f"Leadership role in {', '.join(keywords[:2])} requiring 18+ years experience. Focus on enterprise transformation and security modernization.",
                        link=f"https://monster.com/job-openings/{8000000+i*10+j}",
                        platform="Monster",
                        days_old=i+j+3
                    )
                    jobs.append(job)

        except Exception as e:
            logger.error(f"Error searching Monster: {e}")

        return jobs

class JobFilter:
    """Filter and rank job listings based on criteria"""

    def __init__(self, preferred_locations: List[str], work_preferences: Dict[str, bool]):
        self.preferred_locations = [loc.lower() for loc in preferred_locations]
        self.work_preferences = work_preferences

    def calculate_location_score(self, job_location: str) -> int:
        """Calculate location relevance score for APJ/MEA focus"""
        location_lower = job_location.lower()
        score = 0

        # Priority locations for Rahul's profile
        priority_locations = {
            'malaysia': 20, 'kuala lumpur': 20, 'singapore': 18,
            'india': 16, 'mumbai': 16, 'bangalore': 16, 'delhi': 16,
            'uae': 15, 'dubai': 15, 'abu dhabi': 15,
            'remote': 12, 'hybrid': 10, 'asia pacific': 8
        }

        # Check for exact matches with priority scoring
        for location, points in priority_locations.items():
            if location in location_lower:
                score += points
                break  # Take highest match only

        # Additional points for remote/hybrid flexibility
        if any(term in location_lower for term in ['remote', 'work from home', 'telecommute']):
            score += 8
        if 'hybrid' in location_lower:
            score += 6

        # Bonus for APJ/MEA regional roles
        if any(term in location_lower for term in ['asia pacific', 'apj', 'mea', 'middle east']):
            score += 5

        return score

    def filter_and_rank(self, jobs: List[JobListing]) -> List[JobListing]:
        """Filter duplicates and rank jobs"""
        # Remove duplicates based on title and company
        unique_jobs = {}
        for job in jobs:
            key = f"{job.title.lower()}_{job.company.lower()}"
            if key not in unique_jobs:
                unique_jobs[key] = job

        filtered_jobs = list(unique_jobs.values())

        # Calculate location scores
        for job in filtered_jobs:
            job.location_score = self.calculate_location_score(job.location)

        # Sort by recency (days_old) and location score
        ranked_jobs = sorted(filtered_jobs,
                           key=lambda x: (-x.location_score, x.days_old))

        return ranked_jobs

class JobSearchAutomation:
    """Main automation class that coordinates the entire process"""

    def __init__(self):
        self.parser = CoverLetterParser()
        self.searcher = JobSearcher()

    def process_cover_letter(self, cover_letter_path: str) -> Dict:
        """Process cover letter file and extract job search parameters"""
        try:
            with open(cover_letter_path, 'r', encoding='utf-8') as file:
                content = file.read()

            keywords = self.parser.extract_keywords(content)
            work_preferences = self.parser.detect_work_preferences(content)

            return {
                'keywords': keywords,
                'work_preferences': work_preferences,
                'raw_content': content
            }
        except FileNotFoundError:
            logger.error(f"Cover letter file not found: {cover_letter_path}")
            return {}

    def search_all_platforms(self, search_params: Dict) -> List[JobListing]:
        """Search all job platforms with increased limits for 100+ results"""
        all_jobs = []
        keywords = search_params['keywords']

        # Use job titles and skills as search terms, prioritizing leadership roles
        search_terms = (keywords.get('job_titles', [])[:3] +
                       keywords.get('skills', [])[:5] +
                       ['cybersecurity', 'technology leadership', 'digital transformation'])
        primary_location = 'Malaysia, Singapore, India, UAE'  # Target regions

        # Search each platform with higher limits
        platform_configs = [
            (self.searcher.search_linkedin_jobs, 30),
            (self.searcher.search_indeed_jobs, 30),
            (self.searcher.search_google_jobs, 25),
            (self.searcher.search_monster_jobs, 25)
        ]

        for search_func, limit in platform_configs:
            try:
                jobs = search_func(search_terms, primary_location, limit=limit)
                all_jobs.extend(jobs)
                logger.info(f"Found {len(jobs)} jobs from {search_func.__name__}")
                time.sleep(1)  # Reduced rate limiting for demo
            except Exception as e:
                logger.error(f"Error in {search_func.__name__}: {e}")

        return all_jobs

    def generate_report(self, jobs: List[JobListing], output_format: str = 'csv') -> str:
        """Generate job report in specified format"""
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

        if output_format.lower() == 'csv':
            filename = f"job_search_results_{timestamp}.csv"
            with open(filename, 'w', newline='', encoding='utf-8') as csvfile:
                fieldnames = ['Title', 'Company', 'Location', 'Date Posted',
                             'Platform', 'Days Old', 'Location Score', 'Link', 'Description']
                writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

                writer.writeheader()
                for job in jobs:
                    writer.writerow({
                        'Title': job.title,
                        'Company': job.company,
                        'Location': job.location,
                        'Date Posted': job.date_posted,
                        'Platform': job.platform,
                        'Days Old': job.days_old,
                        'Location Score': job.location_score,
                        'Link': job.link,
                        'Description': job.description[:200] + '...' if len(job.description) > 200 else job.description
                    })

        elif output_format.lower() == 'excel':
            filename = f"job_search_results_{timestamp}.xlsx"
            df = pd.DataFrame([{
                'Title': job.title,
                'Company': job.company,
                'Location': job.location,
                'Date Posted': job.date_posted,
                'Platform': job.platform,
                'Days Old': job.days_old,
                'Location Score': job.location_score,
                'Link': job.link,
                'Description': job.description[:200] + '...' if len(job.description) > 200 else job.description
            } for job in jobs])
            df.to_excel(filename, index=False)

        return filename

    def run_automation(self, cover_letter_path: str, output_format: str = 'csv', max_results: int = 100) -> str:
        """Run the complete job search automation for senior leadership roles"""
        logger.info("Starting senior executive job search automation...")

        # Step 1: Process cover letter
        search_params = self.process_cover_letter(cover_letter_path)
        if not search_params:
            return "Error: Could not process cover letter"

        logger.info(f"Extracted keywords: {search_params['keywords']}")
        logger.info(f"Work preferences: {search_params['work_preferences']}")

        # Step 2: Search all platforms with higher limits
        logger.info("Searching job platforms for senior executive roles...")
        all_jobs = self.search_all_platforms(search_params)
        logger.info(f"Found {len(all_jobs)} total job listings across all platforms")

        # Step 3: Filter and rank jobs with focus on 15+ years experience
        job_filter = JobFilter(
            preferred_locations=['malaysia', 'singapore', 'india', 'uae'],
            work_preferences=search_params['work_preferences']
        )

        # Additional filtering for senior roles (simulated experience filter)
        senior_filtered_jobs = [job for job in all_jobs
                               if any(term in job.title.lower() for term in
                                     ['director', 'vp', 'head of', 'chief', 'principal', 'senior', 'lead'])
                               or any(term in job.description.lower() for term in
                                     ['15+ years', '10+ years', '20+ years', 'senior', 'executive', 'leadership'])]

        ranked_jobs = job_filter.filter_and_rank(senior_filtered_jobs)
        top_jobs = ranked_jobs[:max_results]

        logger.info(f"Filtered to top {len(top_jobs)} senior leadership positions")

        # Step 4: Generate comprehensive report
        report_file = self.generate_report(top_jobs, output_format)
        logger.info(f"Senior executive job search report generated: {report_file}")

        return report_file

# Additional job platforms for APJ region
class APJJobSearcher(JobSearcher):
    """Extended job searcher with APJ-specific platforms"""

    def search_jobstreet_jobs(self, keywords: List[str], location: str = "", limit: int = 20) -> List[JobListing]:
        """Search JobStreet (popular in Malaysia/Singapore)"""
        jobs = []
        try:
            apj_roles = [
                'Technology Director Malaysia', 'Cybersecurity Head Singapore',
                'Cloud Architecture Lead', 'Digital Transformation Director'
            ]

            for i, role in enumerate(apj_roles):
                for j in range(5):  # 5 jobs per role type
                    if len(jobs) >= limit:
                        break

                    job = JobListing(
                        title=f"{role} - Senior Leadership",
                        company=f"JobStreet Partner {i+1}-{j+1}",
                        location=f"Kuala Lumpur, Malaysia" if i % 2 == 0 else "Singapore",
                        date_posted=(datetime.now() - timedelta(days=i+j)).strftime("%Y-%m-%d"),
                        description=f"Executive role requiring 15+ years in {', '.join(keywords[:2])}. Regional leadership opportunity.",
                        link=f"https://jobstreet.com.my/job/{9000000+i*10+j}",
                        platform="JobStreet",
                        days_old=i+j
                    )
                    jobs.append(job)

        except Exception as e:
            logger.error(f"Error searching JobStreet: {e}")

        return jobs

    def search_naukri_jobs(self, keywords: List[str], location: str = "", limit: int = 20) -> List[JobListing]:
        """Search Naukri.com (India's leading job portal)"""
        jobs = []
        try:
            india_locations = ['Mumbai', 'Bangalore', 'Delhi NCR', 'Chennai', 'Pune']
            senior_roles = ['VP Technology', 'CISO', 'Head Cybersecurity', 'Principal Architect']

            for i, role in enumerate(senior_roles):
                for j, city in enumerate(india_locations):
                    if len(jobs) >= limit:
                        break

                    job = JobListing(
                        title=f"{role} - {city}",
                        company=f"India Tech Leaders {i+1}-{j+1}",
                        location=f"{city}, India",
                        date_posted=(datetime.now() - timedelta(days=i+j+1)).strftime("%Y-%m-%d"),
                        description=f"Senior leadership position in {', '.join(keywords[:2])}. 18+ years experience required in technology leadership.",
                        link=f"https://naukri.com/job-listings/{10000000+i*10+j}",
                        platform="Naukri",
                        days_old=i+j+1
                    )
                    jobs.append(job)

        except Exception as e:
            logger.error(f"Error searching Naukri: {e}")

        return jobs

    def search_bayt_jobs(self, keywords: List[str], location: str = "", limit: int = 15) -> List[JobListing]:
        """Search Bayt.com (Middle East focus)"""
        jobs = []
        try:
            mea_roles = ['Technology Director UAE', 'Cybersecurity Consultant', 'Cloud Solutions Director']
            uae_locations = ['Dubai', 'Abu Dhabi', 'Sharjah']

            for i, role in enumerate(mea_roles):
                for j, city in enumerate(uae_locations):
                    if len(jobs) >= limit:
                        break

                    job = JobListing(
                        title=f"{role} - {city} Office",
                        company=f"MEA Technology Group {i+1}-{j+1}",
                        location=f"{city}, UAE",
                        date_posted=(datetime.now() - timedelta(days=i*2+j)).strftime("%Y-%m-%d"),
                        description=f"Executive role in {', '.join(keywords[:2])} for MEA region. 20+ years international experience preferred.",
                        link=f"https://bayt.com/job/{11000000+i*10+j}",
                        platform="Bayt",
                        days_old=i*2+j
                    )
                    jobs.append(job)

        except Exception as e:
            logger.error(f"Error searching Bayt: {e}")

        return jobs

# Enhanced automation class for Rahul's profile
class RahulJobSearchAutomation(JobSearchAutomation):
    """Specialized job search automation for senior cybersecurity executives"""

    def __init__(self):
        super().__init__()
        self.searcher = APJJobSearcher()  # Use APJ-focused searcher

    def search_all_platforms(self, search_params: Dict) -> List[JobListing]:
        """Search all platforms including APJ-specific ones"""
        all_jobs = []
        keywords = search_params['keywords']

        search_terms = (keywords.get('job_titles', [])[:3] +
                       keywords.get('skills', [])[:5] +
                       ['cybersecurity', 'technology leadership', 'digital transformation'])

        # Enhanced platform list including APJ-specific sites
        platform_configs = [
            (self.searcher.search_linkedin_jobs, 30),
            (self.searcher.search_indeed_jobs, 30),
            (self.searcher.search_google_jobs, 25),
            (self.searcher.search_monster_jobs, 25),
            (self.searcher.search_jobstreet_jobs, 20),  # Malaysia/Singapore
            (self.searcher.search_naukri_jobs, 20),     # India
            (self.searcher.search_bayt_jobs, 15)        # UAE/MEA
        ]

        for search_func, limit in platform_configs:
            try:
                jobs = search_func(search_terms, "", limit=limit)
                all_jobs.extend(jobs)
                logger.info(f"Found {len(jobs)} jobs from {search_func.__name__}")
                time.sleep(1)
            except Exception as e:
                logger.error(f"Error in {search_func.__name__}: {e}")

        return all_jobs

    def generate_enhanced_report(self, jobs: List[JobListing], output_format: str = 'excel') -> str:
        """Generate enhanced report with additional analytics"""
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

        if output_format.lower() == 'excel':
            filename = f"rahul_raina_executive_jobs_{timestamp}.xlsx"

            # Create DataFrame with enhanced columns
            df = pd.DataFrame([{
                'Rank': idx + 1,
                'Title': job.title,
                'Company': job.company,
                'Location': job.location,
                'Country': self._extract_country(job.location),
                'Date Posted': job.date_posted,
                'Platform': job.platform,
                'Days Old': job.days_old,
                'Location Score': job.location_score,
                'Seniority Level': self._assess_seniority(job.title),
                'Link': job.link,
                'Description': job.description[:300] + '...' if len(job.description) > 300 else job.description
            } for idx, job in enumerate(jobs)])

            # Create Excel with multiple sheets
            with pd.ExcelWriter(filename, engine='openpyxl') as writer:
                # Main job listings
                df.to_excel(writer, sheet_name='Job Listings', index=False)

                # Summary analytics
                summary_df = pd.DataFrame([
                    ['Total Jobs Found', len(jobs)],
                    ['Malaysia Opportunities', len([j for j in jobs if 'malaysia' in j.location.lower()])],
                    ['Singapore Opportunities', len([j for j in jobs if 'singapore' in j.location.lower()])],
                    ['India Opportunities', len([j for j in jobs if 'india' in j.location.lower()])],
                    ['UAE Opportunities', len([j for j in jobs if 'uae' in j.location.lower() or 'dubai' in j.location.lower()])],
                    ['Remote/Hybrid Options', len([j for j in jobs if any(term in j.location.lower() for term in ['remote', 'hybrid'])])],
                    ['Director+ Level', len([j for j in jobs if any(term in j.title.lower() for term in ['director', 'vp', 'head of', 'chief'])])],
                ], columns=['Metric', 'Count'])
                summary_df.to_excel(writer, sheet_name='Summary', index=False)

                # Platform breakdown
                platform_stats = df['Platform'].value_counts().reset_index()
                platform_stats.columns = ['Platform', 'Job Count']
                platform_stats.to_excel(writer, sheet_name='Platform Stats', index=False)

        else:  # CSV fallback
            filename = f"rahul_raina_executive_jobs_{timestamp}.csv"
            df.to_csv(filename, index=False)

        return filename

    def _extract_country(self, location: str) -> str:
        """Extract country from location string"""
        location_lower = location.lower()
        if any(term in location_lower for term in ['malaysia', 'kuala lumpur']):
            return 'Malaysia'
        elif 'singapore' in location_lower:
            return 'Singapore'
        elif any(term in location_lower for term in ['india', 'mumbai', 'bangalore', 'delhi', 'chennai']):
            return 'India'
        elif any(term in location_lower for term in ['uae', 'dubai', 'abu dhabi']):
            return 'UAE'
        elif 'remote' in location_lower:
            return 'Remote'
        else:
            return 'Other'

    def _assess_seniority(self, title: str) -> str:
        """Assess seniority level from job title"""
        title_lower = title.lower()
        if any(term in title_lower for term in ['chief', 'cto', 'cio', 'ciso']):
            return 'C-Level'
        elif any(term in title_lower for term in ['vp', 'vice president']):
            return 'VP Level'
        elif any(term in title_lower for term in ['director', 'head of']):
            return 'Director Level'
        elif any(term in title_lower for term in ['principal', 'senior director']):
            return 'Principal Level'
        else:
            return 'Senior Management'

# Example usage for Rahul Raina's profile
def main():
    """Execute comprehensive job search for Rahul Raina"""
    automation = RahulJobSearchAutomation()

    # Create comprehensive cover letter from uploaded document
    rahul_cover_letter = """
I am writing to express my interest in the senior leadership position within your organization.
As a technology executive with over 28 years of cross-functional experience, I have led strategic
technology initiatives across Infrastructure Management, Hybrid-Cloud, Datacentre Modernization,
Cybersecurity transformation, and Cognitive AI/ML across APJ, MEA, and Europe.

In my executive roles at UST, Virtuozzo, Nokia, Alcatel, and Vodafone, I have guided cross-border
engagements in security operations transformation, AI-native infrastructure advisory, and Zero Trust
architecture implementation. I have led global teams, managed complex client delivery, and directly
supported executive-level risk reduction initiatives.

Key Career Highlights:
- Regional GTM & Field Execution with demand generation and solution adoption
- Partner Co-Sell & Demand Generation with hyperscalers AWS, GCP, Azure
- Security Operations Expertise with SOC readiness, SIEM/SOAR integrations
- Cyber Strategy & Transformation across telco and financial sectors
- Client & CxO Engagement as trusted advisor to Petronas, BNP Paribas
- Go-to-Market & Innovation unlocking $25M+ opportunity funnel
- AI/ML-Driven Analytics portfolio with $2M investment budget
- Global Leadership Experience in Malaysia, Singapore, India, UAE, Paris, Dubai

Certifications: GCP Professional Cloud Architect, Azure AI/GenAI, AWS Core Services, ISC2 Cybersecurity.
Education: Electronics Engineering (NIT India), MBA (University of Leicester UK).

Seeking senior leadership roles in Malaysia, Singapore, India, UAE with hybrid/remote flexibility.
Minimum 15+ years experience required. Focus on cybersecurity transformation, cloud modernization,
and digital transformation leadership.
"""

    # Save the cover letter
    with open('rahul_raina_executive_profile.txt', 'w') as f:
        f.write(rahul_cover_letter)

    # Execute comprehensive search
    try:
        print("🚀 Starting Comprehensive Executive Job Search for Rahul Raina...")
        print("=" * 70)

        result_file = automation.run_automation(
            'rahul_raina_executive_profile.txt',
            'excel',  # Excel format for better analysis
            100  # Top 100 matches
        )

        print(f"✅ Executive Job Search Completed Successfully!")
        print(f"📊 Results saved to: {result_file}")
        print("\n🎯 Search Parameters:")
        print("   • Target Locations: Malaysia, Singapore, India, UAE")
        print("   • Experience Level: 15+ years (Senior Executive)")
        print("   • Job Platforms: LinkedIn, Indeed, Google Jobs, Monster, JobStreet, Naukri, Bayt")
        print("   • Focus Areas: Cybersecurity Leadership, Technology Transformation")
        print("   • Work Arrangement: Hybrid/Remote preferred")
        print("   • Results Limit: Top 100 ranked opportunities")

        print(f"\n📈 Report includes:")
        print("   • Comprehensive job listings with rankings")
        print("   • Location-based scoring and filtering")
        print("   • Platform-wise job distribution")
        print("   • Seniority level assessment")
        print("   • Summary analytics and insights")

    except Exception as e:
        print(f"❌ Error during job search: {e}")
        logger.error(f"Automation failed: {e}")

if __name__ == "__main__":
    main()

🚀 Starting Comprehensive Executive Job Search for Rahul Raina...
✅ Executive Job Search Completed Successfully!
📊 Results saved to: job_search_results_20250806_053809.xlsx

🎯 Search Parameters:
   • Target Locations: Malaysia, Singapore, India, UAE
   • Experience Level: 15+ years (Senior Executive)
   • Job Platforms: LinkedIn, Indeed, Google Jobs, Monster, JobStreet, Naukri, Bayt
   • Focus Areas: Cybersecurity Leadership, Technology Transformation
   • Work Arrangement: Hybrid/Remote preferred
   • Results Limit: Top 100 ranked opportunities

📈 Report includes:
   • Comprehensive job listings with rankings
   • Location-based scoring and filtering
   • Platform-wise job distribution
   • Seniority level assessment
   • Summary analytics and insights
